In [2]:
%load_ext autoreload 
%autoreload 2

In [7]:
import pandas as pd 
#import geopandas as gpd 
import os 
import requests
import urllib
import io
import numpy as np 

In [2]:
os.getcwd()

'/Users/max/Documents/dcp-projects/db-equitable-development-tool/notebooks'

In [3]:
import sys 
sys.path.append('../utils')
import wd_management 
wd_management.set_wd_root()

In [4]:
hny_url = "https://data.cityofnewyork.us/api/views/hg8x-zxpr/rows.csv?accessType=DOWNLOAD"

hny = requests.get(hny_url).content

hny_df = pd.read_csv(io.StringIO(hny.decode('utf-8')))



In [5]:
hny_df.shape

(5362, 41)

#### Lets do some exploratory analysis to understand the data 
* We are particularily interested in the amount of records with an address but no lat long data in the respective columns - how much work will we need to do to geocode these records on our end?



In [8]:
# number and street are encoded with ---- insted of nan
hny_df = hny_df.replace('----', np.nan)

In [11]:
# how many records have any NANA 
#hny_df_na = hny_df[hny_df.isna().any(axis=1)]

#hny_df_na.shape


(3557, 41)

In [9]:
def show_missing(df):
    """
    Return the total missing values and the percentage of
    missing values by column. 
    """
    null_count = df.isnull().sum()
    null_percentage = (null_count / df.shape[0]) * 100
    return pd.DataFrame({'count_missing_records': null_count, 'missing_percentage': null_percentage})

In [10]:
show_missing(hny_df)

,count_missing_records,missing_percentage
Project ID,0,0.000000
Project Name,0,0.000000
Project Start Date,0,0.000000
Project Completion Date,1608,29.988810
Building ID,1021,19.041402
Number,1021,19.041402
Street,1021,19.041402
Borough,0,0.000000
Postcode,1083,20.197687
BBL,1154,21.521820


In [11]:
hny_missing_geo = hny_df.loc[(hny_df['Latitude'].isnull()) & (hny_df['Street'].notnull())]

In [12]:
hny_missing_geo.shape

(62, 41)

In [13]:
from IPython.display import display 
pd.options.display.max_columns = None

In [14]:
display(hny_missing_geo)

,Project ID,Project Name,Project Start Date,Project Completion Date,Building ID,Number,Street,Borough,Postcode,BBL,BIN,Community Board,Council District,Census Tract,NTA - Neighborhood Tabulation Area,Latitude,Longitude,Latitude (Internal),Longitude (Internal),Building Completion Date,Reporting Construction Type,Extended Affordability Only,Prevailing Wage Status,Extremely Low Income Units,Very Low Income Units,Low Income Units,Moderate Income Units,Middle Income Units,Other Income Units,Studio Units,1-BR Units,2-BR Units,3-BR Units,4-BR Units,5-BR Units,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units
3,44223,ROCHESTER SUYDAM PHASE 1,06/30/2021,NaN,977564.0,329/331,RALPH AVENUE,Brooklyn,NaN,NaN,NaN,BK-03,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New Construction,No,Non Prevailing Wage,0,0,0,10,0,0,0,3,7,0,0,0,0,0,0,10,10,10
160,70806,1718 PRESIDENT STREET,06/29/2021,NaN,998916.0,1781,PRESIDENT STREET,Brooklyn,NaN,NaN,NaN,BK-09,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New Construction,No,Non Prevailing Wage,0,0,10,0,0,0,2,8,0,0,0,0,0,0,10,0,10,46
194,69140,FORDHAM BEDFORD.FB CLUSTER.NR REHAB,06/25/2021,NaN,122133.0,17,WEST MOSHOLU PARKWAY NORTH,Bronx,NaN,NaN,NaN,BX-07,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Preservation,No,Non Prevailing Wage,25,0,24,0,0,1,0,39,7,4,0,0,0,0,50,0,50,50
206,69285,3800 PUTNAM AVENUE GARDENS HDFC.HPO.FY21,06/23/2021,06/23/2021,863998.0,3800,PUTNAM AVENUE WEST,Bronx,NaN,NaN,NaN,BX-08,11.0,NaN,NaN,NaN,NaN,NaN,NaN,06/23/2021,Preservation,Yes,Non Prevailing Wage,0,0,44,0,0,0,0,31,13,0,0,0,0,0,44,0,44,44
257,69331,3-11 27TH AVENUE,05/14/2021,NaN,993634.0,11-Mar,27 AVENUE,Queens,NaN,NaN,NaN,QN-01,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New Construction,No,Non Prevailing Wage,0,0,24,0,0,0,5,6,13,0,0,0,0,0,24,0,24,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,53017,Nehemiah Spring Creek Homes at Gateway Estates 4A,06/27/2016,05/28/2019,972193.0,503,SCHROEDERS AVENUE,Brooklyn,NaN,3.044520e+09,NaN,BK-05,42.0,NaN,NaN,NaN,NaN,40.65486,-73.873762,03/15/2019,New Construction,No,Non Prevailing Wage,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1
3678,53017,Nehemiah Spring Creek Homes at Gateway Estates 4A,06/27/2016,05/28/2019,972194.0,505,SCHROEDERS AVENUE,Brooklyn,NaN,3.044520e+09,NaN,BK-05,42.0,NaN,NaN,NaN,NaN,40.65488,-73.873697,04/12/2019,New Construction,No,Non Prevailing Wage,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1
3679,53017,Nehemiah Spring Creek Homes at Gateway Estates 4A,06/27/2016,05/28/2019,972195.0,507,SCHROEDERS AVENUE,Brooklyn,NaN,3.044520e+09,NaN,BK-05,42.0,NaN,NaN,NaN,NaN,40.65493,-73.873596,04/23/2019,New Construction,No,Non Prevailing Wage,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1
3944,49635,Habitat-NYC Single Family Homes Phase I,12/31/2015,05/31/2016,607523.0,137-35,220 PLACE,Queens,NaN,NaN,NaN,QN-13,31.0,NaN,NaN,NaN,NaN,NaN,NaN,01/08/2015,Preservation,No,Non Prevailing Wage,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1


#### Conclusion 

There are only 62 records out of 5362 that have a missing lat/long but have some sort of street address. From a quick run through of these records, there look to be some encoding errors for certain records (house #'s becing encoded as dates i.e 3-11 27TH AVENUE becoming -->	11-Mar	27 AVENUE), some may also be from simple spelling errors that HPD geocoding wasn't able to catch. Can write this list and send back to HPD to rectify errors - Amanda doesn't think this will be too big of an issue. 

Although over 20% of records have missing lat/long and/or address level information, HPD provides us with the CD which can  be roughly mapped to PUMA level geography which for our purposes should be sufficient for the EDTT project.

